In [1]:
import bs4
import csv
import requests
import re
from concurrent.futures import ThreadPoolExecutor
from tqdm.notebook import tqdm
import random
import pickle
import time

reviewErrorCount = [] # used in multithreadwritecsvfile
dataDir = './data/'

In [2]:
def loadRawData(*args):
    allPagesRequest = []
    for arg in args:
        with open(dataDir+arg,'rb') as file:
            allPagesRequest = allPagesRequest+ pickle.load(file)
    return allPagesRequest

    
    
allPagesRequest = loadRawData('ny-emalje','iwao')

In [3]:
#Frasortere emojis
def remove_emojis(data):
    emoj = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
                      "]+", re.UNICODE)
    return re.sub(emoj, '', data)

#Frasortere linjeskift, comma, og dobbelt mellemrum & gør al tekst lowercase.
def data_cleaning(data):
    data = data.replace(",", " ")
    data = data.replace(' "" ', "")
    data = data.replace('  ', "")
    data = data.replace(' \n ', "")
    data = data.lower()
    return data


#Splitter hver url i listen, og returnere hjemmesidenavnet.
def createFiles(url):
    filename = str(url.url)
    filename = filename.split('.')

    if len(filename) == 4:
        filename = filename[2].split('/')
        filename = filename[2]
    else:
        filename = filename[3]
    return filename


#Tjekker om filen eksistere, så den ikke bliver appended til igen. 
def checkFileExists(filename):
    
    try:
        with open(filename,'rb') as file:
            return 0
    except:
        return 1


print("OK")

OK


In [4]:
#Filtere den gå data med bs4 og skriver til en csv med det navn som request url'et indholder.
fileAlreadyExists = 0  
def multithreadwritecsvfile(page):
    loadingbar.update(1)
    
    filename = dataDir+createFiles(page)+'.csv'
    
    fileAlreadyExists = checkFileExists(filename)
    if fileAlreadyExists == 1:    
        with open(filename, 'a', newline='', encoding='utf-8') as output_file:
            output_writer = csv.writer(output_file)

            output_writer.writerow(['rating','review'])

            soup = bs4.BeautifulSoup(page.content,'html.parser')
            for content in soup.find_all('section', attrs={'class':'styles_reviewContentwrapper__zH_9M'}):
                rating=None
                splitted=None
                review=None

                try: 
                    rating = content.find('img',alt = True)
                    splitted = rating.get('alt').split()
                except Exception as e:
                    print('No rating found. Skipping...', e)

                try: 
                    review = content.find('p', attrs={'class':'typography_typography__QgicV typography_body__9UBeQ typography_color-black__5LYEn typography_weight-regular__TWEnf typography_fontstyle-normal__kHyN3'}).text
                    if len(review) > 2:
                        review = remove_emojis(review)
                        review = data_cleaning(review)
                        output_writer.writerow(['__label__'+splitted[2]+" ", review])
                except Exception as e:
                    reviewErrorCount.append(e);            

  
with tqdm(total=len(allPagesRequest)) as loadingbar: 
    with ThreadPoolExecutor(4) as ex:
        
        ex.map(multithreadwritecsvfile, allPagesRequest)

print('Done. Skipped reviews:', len(reviewErrorCount))



Done. Skipped reviews: 0
